# Forward kinematics for the modified Tinkerkit Braccio

## Define the basic DH transformation matrix

In [21]:
from sympy import *
# define the variables we are using within 
var('d,a,alpha,delta,Rz,Rx,Tz,Tx')

Rx=Matrix([[1,0,0,0],
          [0,cos(alpha),-sin(alpha),0],
          [0,sin(alpha),cos(alpha),0],
          [0,0,0,1]])
Rz=Matrix([[cos(delta),-sin(delta),0,0],
          [sin(delta),cos(delta),0,0],
          [0,0,1,0],
          [0,0,0,1]])
Tz=Matrix([[1,0,0,0],
          [0,1,0,0],
          [0,0,1,d],
          [0,0,0,1]])
Tx=Matrix([[1,0,0,a],
          [0,1,0,0],
          [0,0,1,0],
          [0,0,0,1]])
TM=Tz.multiply(Rz).multiply(Tx).multiply(Rx)
TM

Matrix([
[cos(delta), -sin(delta)*cos(alpha),  sin(alpha)*sin(delta), a*cos(delta)],
[sin(delta),  cos(alpha)*cos(delta), -sin(alpha)*cos(delta), a*sin(delta)],
[         0,             sin(alpha),             cos(alpha),            d],
[         0,                      0,                      0,            1]])

## Definition of the transformation function for each link

In [22]:
# function for converting degrees to radiant
def d2r(deg):
    return deg/180*pi

def Tv5n4(delta5):
    var('delta5,d5,alpha5,a5')
    #DH-Parameter
    d5=150 
    #convert angle from degree to rad and compensate the offset from the home position
    delta5=d2r(delta5) - pi/2 + 0
    a5=0    
    alpha5=0
    return TM.subs({delta:delta5,d:d5,alpha:alpha5,a:a5})

def Tv4n3(delta4):
    var('delta4,d4,alpha4,a4')
    #DH-Parameter
    d4=0
    #convert angle from degree to rad and compensate the offset from the home position
    delta4= d2r(delta4) +0 - pi/2 
    a4=90    
    alpha4=-pi/2
    return TM.subs({delta:delta4,d:d4,alpha:alpha4,a:a4})

def Tv3n2(delta3):
    var('delta3,d3,alpha3,a3')
    #DH-Parameter
    d3=0
    #convert angle from degree to rad and compensate the offset from the home position
    delta3=d2r(delta3) +pi/2 - pi 
    a3=125    
    alpha3=0
    return TM.subs({delta:delta3,d:d3,alpha:alpha3,a:a3})

def Tv2n1(delta2):
    var('delta2,d2,alpha2,a2')
    #DH-Parameter
    d2=0
    #convert angle from degree to rad and compensate the offset from the home position
    delta2= d2r(delta2) + pi/2 - pi/2
    a2=125    
    alpha2=0
    return TM.subs({delta:delta2,d:d2,alpha:alpha2,a:a2})

def Tv1n0(delta1):
    var('delta1,d1,alpha1,a1')
    #DH-Parameter
    d1=70
    #convert angle from degree to rad and compensate the offset from the home position
    delta1=d2r(delta1) + pi - 0
    a1=0    
    alpha1=pi/2
    return TM.subs({delta:delta1,d:d1,alpha:alpha1,a:a1})

# Test
P5=Matrix([0,0,0,1])
P0=Tv1n0(0)*Tv2n1(90)*Tv3n2(180)*Tv4n3(0)*Tv5n4(0)*P5
N(P0)


Matrix([
[275.0],
[    0],
[285.0],
[  1.0]])

## Function for calculating the TCP-position from the joint angles using the forward transformation

In [23]:
def BraccioForward(base, shoulder, elbow,wrist,twist):
    return Tv1n0(base)*Tv2n1(shoulder)*Tv3n2(elbow)*Tv4n3(wrist)*Tv5n4(twist)

# Test
P0=BraccioForward(0,90,0,90,0)*Matrix([0,0,0,1])
N(P0)

Matrix([
[-215.0],
[     0],
[ 345.0],
[   1.0]])

## Test the forward transformation with the Braccio

In [24]:
import serial
import time
s = serial.Serial('COM6', 115200, timeout=5) #zu AHuse: COM3
time.sleep(3)

Move to the Home position

In [25]:
s.write(b'P0,90,180,90,0,50,50\n')
print(s.readline().decode())

OK



Define a function which calculates the gripper's tip position and actuates the robot with the given angles

In [7]:
def TestBraccioForward(base, shoulder, elbow,wrist,twist):
    P0=BraccioForward(base, shoulder, elbow,wrist,twist)*Matrix([0,0,0,1])
    print(pretty(N(P0)))
    command="P"+str(int(base))+"," \
                +str(int(shoulder))+"," \
                +str(int(elbow))+","\
                +str(int(wrist))+","\
                +str(int(twist))+",130,50\n"
    s.write(command.encode('ascii'))
    return #Tv1n0(base)*Tv2n1(shoulder)*Tv3n2(elbow)
    


Put in values for the joint angles, the robot should move to the calculated position. Check with the caliper, if the position is correct.

In [26]:
TestBraccioForward(0,90,0,90,0)

⎡-215.0⎤
⎢      ⎥
⎢  0   ⎥
⎢      ⎥
⎢345.0 ⎥
⎢      ⎥
⎣ 1.0  ⎦


# Inverse Kinematics for the Braccio

## Import additional libraries for inverse kinematics

In [ ]:
import numpy as np
from scipy.optimize import fsolve, minimize
import math

## Analytical Inverse Kinematics Solution

In [ ]:
def BraccioInverse(x, y, z, twist=0):
    """
    逆向运动学函数
    输入：目标位置 (x, y, z) 和 twist角度（默认为0）
    输出：(base, shoulder, elbow, wrist, twist) 角度值（度）
    """
    
    # 机械臂参数（从DH参数中提取）
    d1 = 70   # base高度
    a2 = 125  # shoulder连杆长度
    a3 = 125  # elbow连杆长度
    a4 = 90   # wrist连杆长度
    d5 = 150  # 末端执行器长度
    
    # 1. 计算base角度
    # 由于正向运动学中base有180度偏移，需要调整
    base = math.atan2(-y, -x) * 180 / math.pi
    
    # 2. 计算在xz平面上的投影距离
    r = math.sqrt(x**2 + y**2)
    
    # 3. 调整z坐标（考虑base高度）
    z_adjusted = z - d1
    
    # 4. 计算目标点在2D平面的位置
    # 考虑末端执行器是垂直的情况
    target_r = r
    target_z = z_adjusted - d5
    
    # 5. 使用几何方法求解2R机械臂问题
    # 计算到达目标点的距离
    D = math.sqrt((target_r - a4)**2 + target_z**2)
    
    # 检查是否可达
    if D > (a2 + a3) or D < abs(a2 - a3):
        print(f"目标位置不可达！距离: {D}, 可达范围: [{abs(a2-a3)}, {a2+a3}]")
        return None
    
    # 使用余弦定理计算elbow角度
    cos_theta3 = (D**2 - a2**2 - a3**2) / (2 * a2 * a3)
    cos_theta3 = max(-1, min(1, cos_theta3))  # 限制在[-1, 1]范围内
    
    # 选择elbow-up配置
    theta3 = math.acos(cos_theta3)
    
    # 计算shoulder角度
    k1 = a2 + a3 * math.cos(theta3)
    k2 = a3 * math.sin(theta3)
    gamma = math.atan2(target_z, target_r - a4)
    theta2 = gamma - math.atan2(k2, k1)
    
    # 计算wrist角度以保持末端执行器垂直
    theta4 = -(theta2 + theta3)
    
    # 转换为度数并调整到机械臂的参考系
    shoulder = 90 - theta2 * 180 / math.pi
    elbow = 180 - theta3 * 180 / math.pi
    wrist = 90 - theta4 * 180 / math.pi
    
    # 确保角度在合理范围内
    shoulder = max(0, min(180, shoulder))
    elbow = max(0, min(180, elbow))
    wrist = max(0, min(180, wrist))
    
    return (base, shoulder, elbow, wrist, twist)

## Numerical Inverse Kinematics Solution (Backup)

In [ ]:
def BraccioInverseNumeric(x, y, z, twist=0, initial_guess=None):
    """
    使用数值方法的逆向运动学函数（备选方案）
    适用于解析解失败的情况
    """
    
    def objective(angles):
        """目标函数：最小化末端执行器位置误差"""
        base, shoulder, elbow, wrist = angles
        # 计算正向运动学
        T = BraccioForward(base, shoulder, elbow, wrist, twist)
        P = T * Matrix([0, 0, 0, 1])
        px, py, pz = float(P[0]), float(P[1]), float(P[2])
        # 计算误差
        error = (px - x)**2 + (py - y)**2 + (pz - z)**2
        return error
    
    # 初始猜测
    if initial_guess is None:
        initial_guess = [0, 90, 90, 90]
    
    # 关节限制
    bounds = [
        (-90, 90),    # base
        (0, 180),     # shoulder
        (0, 180),     # elbow
        (0, 180)      # wrist
    ]
    
    # 优化
    result = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')
    
    if result.success and result.fun < 1.0:  # 误差阈值1mm
        base, shoulder, elbow, wrist = result.x
        return (base, shoulder, elbow, wrist, twist)
    else:
        print(f"数值求解失败，误差: {math.sqrt(result.fun):.2f}mm")
        return None

## Test Inverse Kinematics

In [ ]:
def test_inverse_kinematics():
    """
    测试逆向运动学函数
    """
    print("=== 测试逆向运动学 ===")
    
    # 测试点
    test_points = [
        (-215, 0, 345),  # 原始测试点
        (200, 100, 300),
        (0, 250, 200),
        (-150, -150, 250)
    ]
    
    for x, y, z in test_points:
        print(f"\n目标位置: ({x}, {y}, {z})")
        
        # 使用解析解
        result = BraccioInverse(x, y, z)
        if result:
            base, shoulder, elbow, wrist, twist = result
            print(f"解析解: base={base:.1f}°, shoulder={shoulder:.1f}°, "
                  f"elbow={elbow:.1f}°, wrist={wrist:.1f}°, twist={twist:.1f}°")
            
            # 验证结果
            T = BraccioForward(base, shoulder, elbow, wrist, twist)
            P = T * Matrix([0, 0, 0, 1])
            px, py, pz = float(P[0]), float(P[1]), float(P[2])
            error = math.sqrt((px-x)**2 + (py-y)**2 + (pz-z)**2)
            print(f"验证: 实际位置=({px:.1f}, {py:.1f}, {pz:.1f}), 误差={error:.2f}mm")
            
            # 如果误差太大，尝试数值解
            if error > 10:
                print("误差较大，尝试数值解...")
                result_num = BraccioInverseNumeric(x, y, z)
                if result_num:
                    base, shoulder, elbow, wrist, twist = result_num
                    print(f"数值解: base={base:.1f}°, shoulder={shoulder:.1f}°, "
                          f"elbow={elbow:.1f}°, wrist={wrist:.1f}°, twist={twist:.1f}°")
        else:
            print("解析解失败，尝试数值解...")
            result = BraccioInverseNumeric(x, y, z)
            if result:
                base, shoulder, elbow, wrist, twist = result
                print(f"数值解: base={base:.1f}°, shoulder={shoulder:.1f}°, "
                      f"elbow={elbow:.1f}°, wrist={wrist:.1f}°, twist={twist:.1f}°")

# 运行测试
test_inverse_kinematics()

## Wrapper Function for Robot Control

In [ ]:
def MoveBraccioToPosition(x, y, z, twist=0, use_numeric=False):
    """
    将机械臂移动到指定的3D位置
    参数:
        x, y, z: 目标位置坐标（毫米）
        twist: 末端执行器旋转角度（默认0）
        use_numeric: 是否使用数值解（默认False）
    返回:
        True: 成功
        False: 失败
    """
    print(f"\n移动到位置: ({x}, {y}, {z})")
    
    # 获取逆向运动学解
    if use_numeric:
        result = BraccioInverseNumeric(x, y, z, twist)
    else:
        result = BraccioInverse(x, y, z, twist)
        # 如果解析解失败，自动尝试数值解
        if not result:
            print("解析解失败，自动切换到数值解...")
            result = BraccioInverseNumeric(x, y, z, twist)
    
    if result:
        base, shoulder, elbow, wrist, twist = result
        print(f"关节角度: base={base:.1f}°, shoulder={shoulder:.1f}°, "
              f"elbow={elbow:.1f}°, wrist={wrist:.1f}°, twist={twist:.1f}°")
        
        # 发送命令到机器人
        TestBraccioForward(base, shoulder, elbow, wrist, twist)
        return True
    else:
        print("无法到达目标位置")
        return False

## Demo: Move Robot to Different Positions

In [ ]:
# 演示：移动机器人到不同位置
demo_positions = [
    (200, 0, 300),    # 前方
    (0, 200, 300),    # 左侧
    (-200, 0, 300),   # 后方
    (0, -200, 300),   # 右侧
    (150, 150, 250),  # 左前方低位
]

print("开始演示逆向运动学控制...\n")

# 首先回到home位置
print("回到Home位置...")
TestBraccioForward(0, 90, 180, 90, 0)
time.sleep(2)

# 依次移动到各个位置
for i, (x, y, z) in enumerate(demo_positions):
    print(f"\n--- 位置 {i+1}/{len(demo_positions)} ---")
    if MoveBraccioToPosition(x, y, z):
        time.sleep(3)  # 等待机器人移动完成
    else:
        print(f"跳过位置 ({x}, {y}, {z})")

print("\n演示完成！")

In [20]:
s.close()